In [ ]:
from framework.platform import PlatformManager
from framework.configs import PlatformConfig, EvalConfig
import os


eval_cfg = EvalConfig(
    inception_score=True,
    fid=True,
    kid=True,
    prc=False,
    fid_infinity=False,
    is_infinity=False,
    clean_fid=False,
    clean_kid=False,
    clean_fid_batch_size=32,
    clean_kid_batch_size=32,
)

platform_cfg = PlatformConfig(
    verbose=True,
    cuda=True,
    save_cpu_ram=False,
    compare_real_to_real=True
)

platform_manager = PlatformManager(real_images_path=os.path.join(os.getcwd(), "original_64"),
                                   eval_config=eval_cfg,
                                   platform_config=platform_cfg)

result_dict = platform_manager.calc_metrics()
result_dict.print()
result_dict.write_to_json(file="is_fid_infty.json")

In [ ]:
from framework.platform import PlatformManager
from framework.configs import PlatformConfig, EvalConfig, FeatureExtractor
import os



eval_cfg = EvalConfig(
    prd=False,
    feature_extractor=FeatureExtractor.VGGFaceResNet50,
    c2st_knn=False,
    fid_infinity=False,
    is_infinity=False,
    clean_fid=False,
    clean_kid=False,
    inception_score=False,
    fid=False,
    kid=False,
    prc=True,
    ls=False,
)

platform_cfg = PlatformConfig(
    verbose=True,
    cuda=True,
    save_cpu_ram=False,
    compare_real_to_real=True,
    num_worker=16,
)

platform_manager = PlatformManager(real_images_path=os.path.join(os.getcwd(), "original_64"),
                                   eval_config=eval_cfg,
                                   platform_config=platform_cfg)

result_dict = platform_manager.calc_metrics()
result_dict.print()
result_dict.write_to_json(file="dict/is_vggface.json")

In [3]:
from framework.platform import ResultDict

results = ResultDict({})
results.read_from_json("dict/all_vggface.json")
results.print(round_scores=True)

------------------------------------
Generator Name: cifar-10
------------------------------------
FID Infinity (Approx.): 0.343
IS Infinity (Approx.): 1.0
C2ST Adaptive KNN Accuracy: 0.977
C2ST Adaptive KNN Normalized: 0.046
PRD F8 Max Precision: 0.315
PRD F8 Max Recall: 0.128
------------------------------------
Generator Name: diffusionStyleGAN2
------------------------------------
FID Infinity (Approx.): 0.033
IS Infinity (Approx.): 1.0
C2ST Adaptive KNN Accuracy: 0.629
C2ST Adaptive KNN Normalized: 0.742
PRD F8 Max Precision: 0.772
PRD F8 Max Recall: 0.855
------------------------------------
Generator Name: diffusionVAE
------------------------------------
FID Infinity (Approx.): 0.09
IS Infinity (Approx.): 1.0
C2ST Adaptive KNN Accuracy: 0.69
C2ST Adaptive KNN Normalized: 0.621
PRD F8 Max Precision: 0.976
PRD F8 Max Recall: 0.974
------------------------------------
Generator Name: guidedDiffusion_IP
------------------------------------
FID Infinity (Approx.): 0.004
IS Infinity 

In [1]:
import tensorflow as tf
from keras_vggface.vggface import VGGFace
import keras
import torch
from tqdm import tqdm

from torch.utils.data import DataLoader
from torch.utils.data.sampler import SequentialSampler
from framework.datasets import SimpleDataset

# dummy data
torch_tensor = torch.zeros(100, 3, 224, 224)
dataset = SimpleDataset(torch_tensor)
dataloader = DataLoader(dataset, batch_size=64, sampler=SequentialSampler(dataset))
features = []

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

with tf.device('/CPU:0'):           
    # load VGG Face model
    vggface_model = VGGFace(model='resnet50', include_top=True)
    layer_name = 'avg_pool'
    out = vggface_model.get_layer(layer_name).output
    out2 = vggface_model.get_layer('classifier').output
    # create keras model
    vggface_fe = tf.keras.Model(vggface_model.input, [out,out2])
    for tensor in tqdm(dataloader, ascii=True, desc="[INFO]: Feature Extraction"):
        with tf.device('/GPU:0'):
            # pytorch -> tensorflow
            tf_tensor = tf.convert_to_tensor(tensor.permute(0,2,3,1).numpy())
            feature = vggface_fe(tf_tensor)
        # print(f"Shape: {feature.shape}, Device: {feature.device}")
        # tensorflow -> pytorch
        features.append(torch.from_numpy(feature.numpy()))

features = torch.vstack(features).squeeze()
print(features.size())

[INFO]: Feature Extraction:   0%|          | 0/2 [00:00<?, ?it/s]

: 

: 

In [1]:
import torch
import torch.nn.functional as F
from framework.feature_extractor.vggface_nn_module import VGGFaceNNModule

samples = torch.randn(100, 3, 64, 64)

VGGFaceNNModule

fe = VGGFaceNNModule()
pool, logits = fe(samples)
print(pool.size())
print(logits.size())


torch.Size([100, 2048])
torch.Size([100, 8631])
